In [1]:
# just pathing - ignore this
import os
while  not 'juxtorpus' in os.listdir():
    os.chdir('../')
assert 'juxtorpus' in os.listdir(), f"Working directory should be at juxtorpus. But at {os.getcwd()}"
f"Working directory: {os.getcwd()}"

'Working directory: /Users/hcha9747/workspace/juxtorpus'

# Australian Text Analytics Platform (ATAP)

# Corpus Slicer

## If you only want to use a widgetised version of a pandas dataframe you can pretty much ignore the rest.

# Memory Footprint (Bit of computer science)

Every subcorpus just references its parent corpus following all to way to the root corpus (the original corpus you built)

This allows us to keep the memory footprint of the notebook small so that you can be liberal with your dataset size.

## Detaching a Corpus

You can also detach a subcorpus from its tree.
Why would you want to do this:
1. sometimes you have to - part of the design. (e.g. creating a custom DTM from a subcorpus)
2. mostly revolves around having to do performing operations on a subcorpus to change its state.